<a href="https://colab.research.google.com/github/DoItSon/playdata/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D(tabnet)_%ED%95%B4%EB%B3%B4%EA%B8%B0!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 메모장
- 0.8189379090413184 = 0.5
- 0.8236173245493615 = 0.4
- 0.8236625863676823 = 0.3
- 0.8256464235755943 = 0.35

- 0.823185924948775 머신러닝 결측치
- 0.8247486240372479 내 결측치

- 추후 베이지안 최저화를 통해 최적의 파라미터 찾기!

In [55]:
!pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import random

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectPercentile

from pytorch_tabnet.tab_model  import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.metrics import Metric
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score, r2_score, mean_absolute_error

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [57]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
import optuna

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
DATA_PATH = "/content/drive/MyDrive/"

In [61]:
train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')

In [62]:
categorical_features_enc = ['COMPONENT_ARBITRARY_COMPONENT1',
       'COMPONENT_ARBITRARY_COMPONENT2', 'COMPONENT_ARBITRARY_COMPONENT3',
       'COMPONENT_ARBITRARY_COMPONENT4', 'YEAR_2007', 'YEAR_2008', 'YEAR_2009',
       'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015', 'YEAR_2016', 'YEAR_2017', 'YEAR_2018', 'YEAR_2019',
       'YEAR_2020', 'YEAR_2021', 'YEAR_2022']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features_enc = ['ANONYMOUS_1', 'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B',
       'BA', 'BE', 'CA', 'CO', 'CR', 'CU', 'FE', 'H2O', 'LI', 'MG', 'MN', 'MO',
       'NA', 'NI', 'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'TI', 'V',
       'V40', 'ZN', 'CD', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
       'FTBN', 'FUEL', 'K', 'SOOTPERCENTAGE', 'U100', 'U75', 'U50', 'U25',
       'U20', 'U14', 'U6', 'U4', 'V100', 'COMPONENT_ARBITRARY_COMPONENT1',
       'COMPONENT_ARBITRARY_COMPONENT2', 'COMPONENT_ARBITRARY_COMPONENT3',
       'COMPONENT_ARBITRARY_COMPONENT4', 'YEAR_2007', 'YEAR_2008', 'YEAR_2009',
       'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015', 'YEAR_2016', 'YEAR_2017', 'YEAR_2018', 'YEAR_2019',
       'YEAR_2020', 'YEAR_2021', 'YEAR_2022']

## 전처리

In [63]:
CFG = {
    'n_splits':5,
    'SEED':42
}

- 피처 추가

In [64]:
# features = train[["AL","BA"]].sum(axis=1).reset_index()
# features.columns = ["Index","AL+BA"]

# x = pd.concat([train,features],axis = 1)
# train= x.drop(columns=["Index"])
# train

- 내가 직접한 결측치

In [65]:
# fill_mean_func = lambda x: x.fillna(x.median())
# zx = train.groupby(["SAMPLE_TRANSFER_DAY","COMPONENT_ARBITRARY"]).apply(fill_mean_func)
# xc = zx.groupby(["ANONYMOUS_2","YEAR"]).apply(fill_mean_func)

# xc1 = xc.groupby(["ANONYMOUS_1","ANONYMOUS_2"]).apply(fill_mean_func)

# feat = pd.DataFrame(train["ID"].unique(),columns = ["ID"])

# xc2 = xc1.groupby(["MO"]).apply(fill_mean_func)

# features = feat.merge(xc2,how="left",on="ID")

# train = features

In [66]:
# train.isnull().sum()

- 머신러닝 결측치

In [67]:
# col_lst2 = ["CD","FH2O","FNOX","FOPTIMETHGLY","FOXID","FSO4","FTBN","FUEL","K","SOOTPERCENTAGE","U100","U75","U50","U25","U20","U14","U6","U4","V100"]

In [68]:
# imputer = IterativeImputer(estimator =  GradientBoostingRegressor(),random_state=42)
# tmp = imputer.fit_transform(train[col_lst2])

In [69]:
# x2 = pd.DataFrame(tmp,columns=col_lst2)
# train = train.drop(columns=col_lst2)
# train = pd.concat([train,x2],axis=1)
# train

- 다 0으로 채우기!

In [70]:
train = train.fillna(0)
test = test.fillna(0)

In [71]:
train.isnull().sum().sum()

0

## 데이터 분리

In [72]:
train_X  = train.drop(['ID', 'Y_LABEL'], axis = 1) # 정답값 제외
target  = train['Y_LABEL'] # 정답

test = test.drop(['ID'], axis = 1) # 추론 데이터

In [73]:
train_X

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,...,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN
0,COMPONENT3,1486,2011,7,200,0,3,93,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,154.0,75
1,COMPONENT2,1350,2021,51,375,0,2,19,0,0,...,1.0,2.0,4.0,6.0,216.0,1454.0,0,0.0,44.0,652
2,COMPONENT2,2415,2015,2,200,0,110,1,1,0,...,0.0,0.0,3.0,39.0,11261.0,41081.0,0,0.0,72.6,412
3,COMPONENT3,7389,2010,2,200,0,8,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,133.3,7
4,COMPONENT3,3954,2015,4,200,0,1,157,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,133.1,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,135.4,16
14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,14.5,117.5,1408
14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,54.0,1301
14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,...,0.0,7.0,8.0,100.0,1625.0,18890.0,0,0.0,44.3,652


# 원핫인코딩

In [74]:
enc = OneHotEncoder()

tmp = pd.DataFrame(
    enc.fit_transform(train_X[["COMPONENT_ARBITRARY",'YEAR']]).toarray(),
    columns = enc.get_feature_names_out()
)
train_X = pd.concat([train_X,tmp],axis=1).drop(columns = ["COMPONENT_ARBITRARY",'YEAR'])

tmp1 = pd.DataFrame(
    enc.transform(test[["COMPONENT_ARBITRARY",'YEAR']]).toarray(),
    columns = enc.get_feature_names_out()
)
test = pd.concat([test,tmp1],axis=1).drop(columns = ["COMPONENT_ARBITRARY",'YEAR'])

In [75]:
train_X.shape,target.shape,test.shape

((14095, 70), (14095,), (6041, 36))

## Classifier

In [76]:
train_X_toNum = train_X.to_numpy()

In [77]:
target_toNum = target.to_numpy()

In [105]:
test_toNum = test.to_numpy()

In [79]:
class F1_Score(Metric):
    def __init__(self):
        self._name = "f1"
        self._maximize = True

    def __call__(self, y_true, y_score):
        score = f1_score(y_true, (y_score[:, 1]>0.5)*1)
        return score

In [80]:
patience = 50
cv = KFold(n_splits=CFG['n_splits'], shuffle=True, random_state=CFG['SEED'])

In [81]:
pred_list = []
score_list = []
for tri,vai in cv.split(train_X_toNum):

    x_train = train_X_toNum[tri]
    y_train = target_toNum[tri]

    x_valid = train_X_toNum[vai]
    y_valid = target_toNum[vai]
    
    model = TabNetClassifier(verbose = 1,seed = 42,optimizer_fn=torch.optim.AdamW)
    model.fit(x_train,y_train ,
              eval_set = [(x_train,y_train)],
              patience=100, max_epochs=2000,
              eval_metric=['logloss','f1']) 
    
    pred = model.predict_proba(x_valid)[:,-1] 
    pred_list.append(pred)
    
    score_list.append(model.best_cost)

epoch 0  | loss: 0.62601 | val_0_logloss: 0.42    | val_0_f1: 0.1355  |  0:00:01s
epoch 1  | loss: 0.32897 | val_0_logloss: 0.3004  | val_0_f1: 0.39809 |  0:00:02s
epoch 2  | loss: 0.29169 | val_0_logloss: 0.26161 | val_0_f1: 0.45426 |  0:00:03s
epoch 3  | loss: 0.25051 | val_0_logloss: 0.23881 | val_0_f1: 0.54042 |  0:00:05s
epoch 4  | loss: 0.23416 | val_0_logloss: 0.21027 | val_0_f1: 0.57122 |  0:00:08s
epoch 5  | loss: 0.21475 | val_0_logloss: 0.20283 | val_0_f1: 0.56792 |  0:00:10s
epoch 6  | loss: 0.20083 | val_0_logloss: 0.18863 | val_0_f1: 0.61571 |  0:00:12s
epoch 7  | loss: 0.19655 | val_0_logloss: 0.18462 | val_0_f1: 0.61978 |  0:00:14s
epoch 8  | loss: 0.18923 | val_0_logloss: 0.18079 | val_0_f1: 0.63049 |  0:00:17s
epoch 9  | loss: 0.19071 | val_0_logloss: 0.18347 | val_0_f1: 0.60734 |  0:00:19s
epoch 10 | loss: 0.18482 | val_0_logloss: 0.17913 | val_0_f1: 0.62794 |  0:00:21s
epoch 11 | loss: 0.18488 | val_0_logloss: 0.17891 | val_0_f1: 0.63078 |  0:00:23s
epoch 12 | loss:

In [82]:
np.mean(score_list)
# 0.8852591170415698 tabnet

0.8852591170415698

In [83]:
pred = np.concatenate(pred_list,axis=0)
pred.shape

(14095,)

In [84]:
pred

array([0.36436528, 0.11404085, 0.00048684, ..., 0.00171957, 0.0024068 ,
       0.19218786], dtype=float32)

- 확률 추출!

In [85]:
x_target = pd.DataFrame(pred)
x_target = x_target.squeeze()
x_target

0        0.364365
1        0.114041
2        0.000487
3        0.000839
4        0.000093
           ...   
14090    0.587434
14091    0.101837
14092    0.001720
14093    0.002407
14094    0.192188
Name: 0, Length: 14095, dtype: float32

## Regressor

- train의 부분 X의 예측값을 설명하는 Regressor

In [86]:
test_stage_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN',
       'COMPONENT_ARBITRARY_COMPONENT1', 'COMPONENT_ARBITRARY_COMPONENT2',
       'COMPONENT_ARBITRARY_COMPONENT3', 'COMPONENT_ARBITRARY_COMPONENT4',
       'YEAR_2007', 'YEAR_2008', 'YEAR_2009', 'YEAR_2010', 'YEAR_2011',
       'YEAR_2012', 'YEAR_2013', 'YEAR_2014', 'YEAR_2015', 'YEAR_2016',
       'YEAR_2017', 'YEAR_2018', 'YEAR_2019', 'YEAR_2020', 'YEAR_2021',
       'YEAR_2022']

In [90]:
train_X = pd.DataFrame(train_X_toNum,columns = train_X.columns)
train_X

,ANONYMOUS_1,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,...,YEAR_2013,YEAR_2014,YEAR_2015,YEAR_2016,YEAR_2017,YEAR_2018,YEAR_2019,YEAR_2020,YEAR_2021,YEAR_2022
0,1486.0,7.0,200.0,0.0,3.0,93.0,0.0,0.0,3059.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1350.0,51.0,375.0,0.0,2.0,19.0,0.0,0.0,2978.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2415.0,2.0,200.0,0.0,110.0,1.0,1.0,0.0,17.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7389.0,2.0,200.0,0.0,8.0,3.0,0.0,0.0,1960.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3954.0,4.0,200.0,0.0,1.0,157.0,0.0,0.0,71.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,1616.0,8.0,200.0,0.0,2.0,201.0,1.0,0.0,6.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14091,2784.0,2.0,200.0,0.0,3.0,85.0,0.0,0.0,2945.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14092,1788.0,9.0,550.0,0.0,6.0,0.0,1.0,0.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14093,2498.0,19.0,550.0,0.0,2.0,4.0,0.0,0.0,2244.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
train_reg = train_X[test_stage_features]
train_reg.shape,x_target.shape

((14095, 36), (14095,))

In [92]:
train_reg = train_reg.to_numpy()

In [93]:
x_target = x_target.to_numpy()

In [100]:
x_target = x_target.reshape(-1, 1)

In [108]:
preds = []
score_list = []
for tri,vai in cv.split(train_reg):

    x_train = train_reg[tri]
    y_train = x_target[tri]

    x_valid = train_reg[vai]
    y_valid = x_target[vai]
    
    model = TabNetRegressor(verbose = 1,seed = 42,optimizer_fn=torch.optim.AdamW)
    model.fit(train_reg,x_target ,
              eval_set = [(train_reg,x_target)],
              patience=100, max_epochs=2000,
              eval_metric=['mse'])
    
    score_list.append(model.best_cost)
    
    pred = model.predict(x_valid)
    preds.append(pred)

epoch 0  | loss: 0.56845 | val_0_mse: 0.28189000487327576|  0:00:01s
epoch 1  | loss: 0.06961 | val_0_mse: 0.07322999835014343|  0:00:03s
epoch 2  | loss: 0.06035 | val_0_mse: 0.06103000044822693|  0:00:04s
epoch 3  | loss: 0.05887 | val_0_mse: 0.059059999883174896|  0:00:05s
epoch 4  | loss: 0.05802 | val_0_mse: 0.057360000908374786|  0:00:06s
epoch 5  | loss: 0.05814 | val_0_mse: 0.057429999113082886|  0:00:07s
epoch 6  | loss: 0.05815 | val_0_mse: 0.0573900006711483|  0:00:08s
epoch 7  | loss: 0.05791 | val_0_mse: 0.056940000504255295|  0:00:09s
epoch 8  | loss: 0.05743 | val_0_mse: 0.05696000158786774|  0:00:10s
epoch 9  | loss: 0.05768 | val_0_mse: 0.056790001690387726|  0:00:11s
epoch 10 | loss: 0.05774 | val_0_mse: 0.05691000074148178|  0:00:13s
epoch 11 | loss: 0.05761 | val_0_mse: 0.05694999918341637|  0:00:14s
epoch 12 | loss: 0.05695 | val_0_mse: 0.05697999894618988|  0:00:15s
epoch 13 | loss: 0.05792 | val_0_mse: 0.0568699985742569|  0:00:16s
epoch 14 | loss: 0.05747 | val_

In [109]:
np.mean(score_list)
# 0.05544775

0.05544775

In [113]:
preds = []
score_list = []
for tri,vai in cv.split(train_reg):

    x_train = train_reg[tri]
    y_train = x_target[tri]

    x_valid = train_reg[vai]
    y_valid = x_target[vai]
    
    model = TabNetRegressor(verbose = 1,seed = 42,optimizer_fn=torch.optim.AdamW)
    model.fit(train_reg,x_target ,
              eval_set = [(train_reg,x_target)],
              patience=100, max_epochs=2000,
              eval_metric=['mse'])
    
    score_list.append(model.best_cost)
    
    pred = model.predict(test_toNum)
    preds.append(pred)

epoch 0  | loss: 0.56845 | val_0_mse: 0.28189000487327576|  0:00:01s
epoch 1  | loss: 0.06961 | val_0_mse: 0.07322999835014343|  0:00:02s
epoch 2  | loss: 0.06035 | val_0_mse: 0.06103000044822693|  0:00:03s
epoch 3  | loss: 0.05887 | val_0_mse: 0.059059999883174896|  0:00:04s
epoch 4  | loss: 0.05802 | val_0_mse: 0.057360000908374786|  0:00:05s
epoch 5  | loss: 0.05814 | val_0_mse: 0.057429999113082886|  0:00:06s
epoch 6  | loss: 0.05815 | val_0_mse: 0.0573900006711483|  0:00:07s
epoch 7  | loss: 0.05791 | val_0_mse: 0.056940000504255295|  0:00:08s
epoch 8  | loss: 0.05743 | val_0_mse: 0.05696000158786774|  0:00:09s
epoch 9  | loss: 0.05768 | val_0_mse: 0.056790001690387726|  0:00:10s
epoch 10 | loss: 0.05774 | val_0_mse: 0.05691000074148178|  0:00:12s
epoch 11 | loss: 0.05761 | val_0_mse: 0.05694999918341637|  0:00:13s
epoch 12 | loss: 0.05695 | val_0_mse: 0.05697999894618988|  0:00:14s
epoch 13 | loss: 0.05792 | val_0_mse: 0.0568699985742569|  0:00:15s
epoch 14 | loss: 0.05747 | val_

In [116]:
pred

array([[0.10351481],
       [0.10283794],
       [0.10351481],
       ...,
       [0.09929325],
       [0.08104476],
       [0.07365682]], dtype=float32)

In [114]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [122]:
submit.Y_LABEL = (np.where(pred > 0.5, 1, 0)) # 이것도 추후 조정해보기!

In [123]:
submit

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
6036,TEST_6036,0
6037,TEST_6037,0
6038,TEST_6038,0
6039,TEST_6039,0


In [124]:
submit.Y_LABEL.value_counts()

0    6036
1       5
Name: Y_LABEL, dtype: int64

In [121]:
submit.to_csv('/content/drive/MyDrive/submit.csv', index=False)

- 잡다한 것

In [ ]:
# score = f1_score(y_valid, predict,average="macro")
# score = cross_val_score(y_valid, predict, scoring="f1_macro") 

In [ ]:
# score_list = []
    
# # 모델 학습
# model = LGBMClassifier(random_state=42,n_estimators=1000)
# model.fit(train_X,target , eval_set = [(train_X,target)],early_stopping_rounds=50) 

# pred = model.predict_proba(train_X)[:,1] # 1일 확률
# predict = np.where(pred > 0.35, 1 , 0)
# score = f1_score(target,predict,average="macro") 
# score_list.append(score) 

# score_list

In [ ]:
# tpred_list = []
# score_list = []
# for tri,vai in cv.split(test):

#     x_train = test.iloc[tri]
#     y_train = x_target[tri]

#     x_valid = test.iloc[vai]
#     y_valid = x_target[vai]
    

#     model = lgb.LGBMRegressor(random_state=CFG['SEED'],predict_disable_shape_check=True )
#     model.fit(x_train,y_train , eval_set = [(x_valid,y_valid)],early_stopping_rounds=50) 

#     pred = model.predict(x_valid) # 이것을 변환
#     pred_list.append(pred)

# pred_list